# Crashes on a Street

In [1]:
import pytz
import folium
from folium.plugins import HeatMap

import pandas as pd
from datetime import datetime, timedelta
from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
start_date = pytz.timezone('America/Denver').localize(datetime(2013,1,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2023,3,1))
most_recent_crash = cda.most_recent_crash_timestamp()

In [4]:
query = f"""
select
sr.gid
, sr.lrsroute
, st_length(sr.geom_denver) / 5280 as length_miles
, c.sbi
, c.fatality
, c.pedestrian_ind
, c.bicycle_ind
, c.geo_lon
, c.geo_lat
, c.reported_date
, c.reported_date at time zone 'America/Denver' as reported_date_local
, c.incident_address_corrected

from street_routes sr
inner join crashes c on st_dwithin(sr.geom_denver, c.geom_denver, 25)

-- where sr.gid = 1392 -- Gilpin St
where sr.gid = 4306 -- W 23rd Ave
and reported_date at time zone 'America/Denver' > '{start_date.strftime('%Y-%m-%d')}'
and reported_date at time zone 'America/Denver' < '{end_date.strftime('%Y-%m-%d')}'
and not at_freeway

order by reported_date desc
"""

street_crashes = pd.read_sql(query, cda.conn)

In [5]:
street_crashes.groupby(street_crashes.reported_date_local.dt.year).agg(
    crashes=('reported_date_local', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date_local,,,,,
2013,23,0,0,0,1
2014,17,0,1,1,1
2015,20,1,0,0,2
2016,15,0,1,0,1
2017,27,0,0,2,3
2018,22,0,0,0,0
2019,24,0,0,1,1
2020,13,0,0,1,0
2021,20,0,0,1,1


In [6]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(street_crashes[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [7]:
days_in_data = (end_date - start_date).days

In [8]:
days_between_crashes = days_in_data / len(street_crashes)
days_between_crashes

19.328125

In [9]:
crashes_per_mile_per_week = (len(street_crashes) / street_crashes.iloc[0].length_miles) / (days_in_data/7)
crashes_per_mile_per_week

0.24443886583681154

In [10]:
street_crashes.sort_values(by='geo_lat')

,gid,lrsroute,length_miles,sbi,fatality,pedestrian_ind,bicycle_ind,geo_lon,geo_lat,reported_date,reported_date_local,incident_address_corrected
126,4306,W23RD AV,1.481624,False,False,0.0,0.0,-105.041650,39.751193,2016-06-30 01:02:00+00:00,2016-06-29 19:02:00,N RALEIGH ST / W 23RD AVE
181,4306,W23RD AV,1.481624,False,False,0.0,0.0,-105.041650,39.751193,2013-08-28 14:57:00+00:00,2013-08-28 08:57:00,N RALEIGH ST / W 23RD AVE
24,4306,W23RD AV,1.481624,False,False,0.0,0.0,-105.039317,39.751194,2021-04-06 08:28:00+00:00,2021-04-06 02:28:00,N PERRY ST / W 23RD AVE
53,4306,W23RD AV,1.481624,False,False,0.0,0.0,-105.022071,39.751197,2019-09-29 04:20:00+00:00,2019-09-28 22:20:00,W 23RD AVE / N DECATUR ST
35,4306,W23RD AV,1.481624,False,False,0.0,0.0,-105.022071,39.751197,2020-10-22 19:32:00+00:00,2020-10-22 13:32:00,N DECATUR ST / W 23RD AVE
...,...,...,...,...,...,...,...,...,...,...,...,...
125,4306,W23RD AV,1.481624,False,False,0.0,0.0,-105.024365,39.751275,2016-07-20 23:17:00+00:00,2016-07-20 17:17:00,2945 W 23RD AVE
70,4306,W23RD AV,1.481624,False,False,0.0,0.0,-105.016530,39.751320,2018-12-14 17:35:00+00:00,2018-12-14 10:35:00,I25 N/W 23RD AVE
131,4306,W23RD AV,1.481624,False,False,0.0,0.0,-105.016530,39.751320,2016-01-17 00:19:00+00:00,2016-01-16 17:19:00,I25 HWY / W 23RD AVE
61,4306,W23RD AV,1.481624,False,False,0.0,0.0,-105.015168,39.751784,2019-05-05 18:33:00+00:00,2019-05-05 12:33:00,W 23RD AVE / WB 23RD TO NB I25 RAMP
